# Predition de l'energie des molécules

In [ ]:
#### imporation des librairies

In [25]:
import pandas as pd
import numpy as np
# ase est une librairie pour lire les fichiers .xyz (souvent utilisé pour representer des atomes.) site : https://wiki.fysik.dtu.dk/
from ase.io.xyz import read_xyz
from ase.io import read
from ase.visualize import view
import os 

### Chargement des données 

#### Comment lire un fichier .xyz et quelques foncition utiles

In [29]:
test = read("./train/atoms/train/id_1.xyz")

In [44]:
view(test, viewer='x3d')

#### Listing des infos (fonctions) complémentaires potentiellement interessantes pour une molécule

In [32]:
test.get_all_distances().shape

(19, 19)

In [37]:
test.get_masses()

array([14.007, 12.011, 12.011, 12.011, 12.011, 12.011, 14.007,  1.008,
        1.008,  1.008,  1.008,  1.008,  1.008,  1.008,  1.008,  1.008,
        1.008,  1.008,  1.008])

In [34]:
test.get_chemical_formula(mode='hill', empirical=False)

'C5H12N2'

In [40]:
test.get_global_number_of_atoms()

19

In [39]:
test.get_positions()

array([[ 0.857135,  1.766441,  1.943662],
       [ 0.099524,  0.525085,  1.758627],
       [-0.233978,  0.294832,  0.290564],
       [ 0.740909, -0.375661, -0.628106],
       [-0.519742, -1.108375, -0.204441],
       [-1.656612, -1.365652, -1.170557],
       [ 0.916179,  0.131485, -1.980716],
       [ 1.022234,  1.944131,  2.930746],
       [ 0.353962,  2.557052,  1.549804],
       [-0.841044,  0.528336,  2.354028],
       [ 0.698684, -0.314844,  2.141217],
       [-0.841061,  1.081979, -0.162159],
       [ 1.675159, -0.731132, -0.180879],
       [-0.377184, -1.900361,  0.534083],
       [-1.740849, -0.565663, -1.903374],
       [-2.610844, -1.439292, -0.64818 ],
       [-1.507099, -2.299673, -1.712636],
       [ 0.103102,  0.661861, -2.28071 ],
       [ 1.071134, -0.629252, -2.639089]])

In [41]:
test.get_center_of_mass(scaled=False)

array([0.02960112, 0.01087163, 0.00017221])

In [51]:
test.set_calculator()

In [56]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_charges()
except Exception as e: 
    print(e)
    print("Erreur : Necessite de donner un calculateur à la molécule")

Atoms object has no calculator.
Erreur : Necessite de donner un calculateur à la molécule


In [53]:
test.euler_rotate(phi=0.0, theta=0.0, psi=0.0, center=(0, 0, 0))

In [54]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_dipole_moment()
except : 
    print("Erreur : Necessite de donner un calculateur à la molécule")

Erreur : Necessite de donner un calculateur à la molécule


In [49]:
# Necessite de donner un calculateur à la molécule
try :
    test.get_forces(apply_constraint=True, md=False)
except : 
    print("Erreur : Necessite de donner un calculateur à la molécule")

Erreur : Necessite de donner un calculateur à la molécule


In [50]:
test.get_moments_of_inertia(vectors=False)

array([121.81724473, 305.64416583, 383.56495588])

In [ ]:
###